# Storing Teletry


Recover data from the previous steps, make sure you've been through them and understand what they are:

In [1]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A list of sample messages
%store -r payloads
#A sample message payload
%store -r payload

Create a DynamoDB table to store our messages:

In [2]:
table_name = "bloodpressure{}".format(HM)
table_name

'bloodpressure091714'

In [3]:
!echo Creating table "{table_name}" this takes a few seconds...
table_arn = !aws dynamodb create-table \
  --table-name {table_name} \
  --attribute-definitions \
    AttributeName=name,AttributeType=S \
    AttributeName=time_stamp,AttributeType=N \
  --key-schema AttributeName=name,KeyType=HASH AttributeName=time_stamp,KeyType=RANGE \
  --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
  --query TableDescription.TableArn \
  --output text
table_arn = table_arn.s
!aws dynamodb wait table-exists --table-name {table_name}
!echo Table created "{table_arn}"

Creating table bloodpressure091714 this takes a few seconds...
Table created arn:aws:dynamodb:us-east-1:030555009967:table/bloodpressure091714


Create a rule capturing messages from the topic. The rule conditions are expressed in SQL statements like the one below. Note that they can not only capture but also enrich data, see the [AWS IoT SQL Reference
](https://docs.aws.amazon.com/iot/latest/developerguide/iot-sql-reference.html) for more information.

In [4]:
sql = """
SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{}'
""".format(topic_name)
print(sql)


SELECT name, 
    time_stamp, 
    sistolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM 'notebook-topic'



The rule also requires an IAM Role authorizing it to write data on DynamoDB. The following trust relationship policy states that this role can be assumed by AWS IoT.

In [5]:
# %load iot-trust.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "iot.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Statement': [{'Action': 'sts:AssumeRole',
   'Effect': 'Allow',
   'Principal': {'Service': 'iot.amazonaws.com'}}],
 'Version': '2012-10-17'}

In [7]:
role_name = "bp2dynamodb_role{}".format(HM) 
rule_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://glue-trust.json \
    --query Role.Arn \
    --output text
rule_role_arn = rule_role_arn.s
rule_role_arn

'arn:aws:iam::030555009967:role/bp2dynamodb_role091714'

And the rule policy states that it can insert data into this specific DynamoDB table

In [8]:
import json
rule_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Action": "dynamodb:PutItem",
        "Resource": table_arn
    }]
}
rule_policy = json.dumps(rule_policy)
rule_policy

'{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Action": "dynamodb:PutItem", "Resource": "arn:aws:dynamodb:us-east-1:030555009967:table/bloodpressure091714"}]}'

In [9]:
policy_name="bp2dynamodb_rule_policy{}".format(HM)
policy_arn = !aws iam create-policy \
  --query Policy.Arn \
  --output text \
  --policy-name '{policy_name}' \
  --policy-document '{rule_policy}'
policy_arn = policy_arn.s
policy_arn

'arn:aws:iam::030555009967:policy/bp2dynamodb_rule_policy091714'

In [10]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn {policy_arn}

In [11]:
rule = {
    "ruleDisabled": False,
    "sql": sql,
    "description": "Send blood preasure measurements to DynamoDB",
    "actions": [{
        "dynamoDBv2": {
            "roleArn": rule_role_arn,
            "putItem": {
                "tableName": table_name
            }
        }
    }]
}
rule = json.dumps(rule)
rule_file="{}/rule.json".format(home)
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

'/Users/faermanj/rule.json'

In [ ]:
print(open(rule_file).read())

In [12]:
rule_name="bprule{}".format(HM)
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}

In [13]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(HM)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [17]:
import random
payload = random.choice(payloads)
print(payload)
mqtt.publish(topic_name, payload , QoS = 0)

{"id": "e3b0e9d8-78ae-448e-b7dc-7ad43ea8e24b", "name": "Clara Oswald", "sistolic": 147, "diastolic": 107, "time_stamp": 1526887235209}


True

Lets check how many records have arrived on DynamoDB...

In [19]:
!aws dynamodb scan --table-name {table_name} --query Count

0
[Errno 60] Operation timed out
[Errno 54] Connection reset by peer


In [ ]:
mqtt.disconnect()

All right, but what if someone has high blood pressure, better let them know, right?
Let's do that in the [Reacting to Events notebook](aws-iot-react-events.ipynb)